# Solving Machathon 1.0 Competetion

In [89]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

Importing Data from drive since i am using google colab

In [90]:
data = pd.read_csv('/content/drive/MyDrive/X_train1.csv')
data.head(5)

,user_uuid,restaurant_uuid,discounted,area,nOrders,cuisine,restaurant_features,Ratings
0,4f0181917069f1bf5259c04f14a944c49d461bab,bf5ee7c8457db4d32da6633baf7a047115d76af4,0,Heliopolis,1,['Egyptian' 'Oriental' 'Grill'],['No Smoking Area' 'Outdoor' 'TV' 'WiFi' 'Gril...,5
1,06c7a3629566f04e19611981d0fb30dabe8d8daa,a8f1c42a5b27f032bbd69821b3d9941939db1ecf,0,Haram,1,['Burgers' 'Sandwiches' 'Fast Food'],['No Smoking Area' 'Outdoor' 'WiFi' 'Kids-Frie...,4
2,966acbc0eaf5c2cda02a242ec9caf75f98119ef9,ef29eff86405a92b1b547b32e845497694f450c6,0,Maadi,3,['Egyptian' 'Oriental' 'Grill'],['No Smoking Area' 'Khorogat Iftar' 'Catering'],4
3,a910926f38c18c2e00a338c7d2f6e03ee78ea7a4,66ae2fbb6d1dfc1d3b07a68839db5da163eabe68,0,Faisal,1,['Pizza' 'Fast Food'],['24 hrs'],1
4,aa25b862b3f7fbd71425170343c53dcd93ec10c9,ef29eff86405a92b1b547b32e845497694f450c6,0,Maadi,1,['Egyptian' 'Oriental' 'Grill'],['No Smoking Area' 'Khorogat Iftar' 'Catering'],4


Features are crammed up in a single string in the cuisine and restaurant columns. So we should split the whole string into an array of features in order to get their dummy values latter

In [91]:
def translate_feature(s): #splittin the string values
  s = s.split("'")
  s.remove('[')
  s.remove(']')
  s = [x for x in s if x.strip()]
  return s

data['cuisine'] = data['cuisine'].apply(lambda x: translate_feature(x) if not ('array(' in x) else np.nan)
data['restaurant_features'] = data['restaurant_features'].apply(lambda x: translate_feature(x) if not ('array(' in x) else np.nan)

Removing data that were entered incorrectly

In [92]:
data = data.dropna(axis = 0)
data.tail(10)

,user_uuid,restaurant_uuid,discounted,area,nOrders,cuisine,restaurant_features,Ratings
79384,fb4ff935a1ccd659013a9481832d616e15ef5436,a8f1c42a5b27f032bbd69821b3d9941939db1ecf,1,Nasr City,4,"[Burgers, Sandwiches, Fast Food]","[No Smoking Area, Outdoor, WiFi, Kids-Friendly...",5
79385,a51287bd4ab877a917317656295fa5dedfb78050,c5a1c4a17944574b766b5bb538fa1e7a8467b7a7,0,Zamalek,4,[Foul we Ta3meya],"[Outdoor, TV, 24 hrs, Breakfast, Cheap Eats]",3
79386,4c99a553379a8772611399349b6a4bc13f6ed0f5,44c1d7e1423726fc18015bc0314f3f8f239ee3e1,1,New Cairo,1,"[Sandwiches, Syrian]","[Outdoor, TV, Cheap Eats]",4
79387,1da8bf20b5e7bb4384743f33dbf4d33fa0232b7d,9d598592df10654914c702db2864bbcf36ea2918,0,Mohandessin,1,"[Sandwiches, Fast Food]","[No Smoking Area, TV, Khorogat Iftar]",1
79389,79cf445226e6d0b4ecbd341323263880ffc4cb4f,c7a21ff3fae177fe5b5ce7e768b4b303af1a48ed,0,New Cairo,1,"[Asian, Chinese, Fast Food]","[No Smoking Area, Casual Dining, Outdoor, TV]",5
79390,03406503f7701dc5fde1887170f13aac4d75d333,c2e51bf525d2b2c7a33c3d8ee017914c6400e975,0,Nasr City,3,"[Egyptian, Fast Food]",[No Smoking Area],5
79391,da8d4dbcf05a79f452f24494e4375f0c768a0d13,5f05a1ca558b36c593f2ab9f552f7fadbed9c383,0,Heliopolis,1,"[Asian, Japanese, Sushi, Steaks]","[No Smoking Area, Casual Dining, Outdoor, WiFi...",5
79392,0fcd18e59f31106fd855b6cf340bf00bde6d6f99,c2e51bf525d2b2c7a33c3d8ee017914c6400e975,1,Maadi,2,"[Egyptian, Fast Food]",[No Smoking Area],5
79393,cf77a30aa4de6e8dfc940ddea222a84766de925f,bf5ee7c8457db4d32da6633baf7a047115d76af4,0,New Cairo,3,"[Egyptian, Oriental, Grill]","[No Smoking Area, Outdoor, TV, WiFi, Grill, Ca...",4
79394,89f5477912a0101d0a58a1c21efbc03577f7016c,bf5ee7c8457db4d32da6633baf7a047115d76af4,0,Mohandessin,1,"[Egyptian, Oriental, Grill]","[No Smoking Area, Outdoor, TV, WiFi, Grill, Ca...",5


Creatting a Set object to get all uniques features

In [93]:
cuisine_features = {'set_init'}
restaurant_features = {'set_init'}

def bag_of_cuisines(s, bag):
  for i in s:
    bag.add(i)

  return s

data['cuisine'] = data['cuisine'].apply(lambda x: bag_of_cuisines(x, cuisine_features))
data['restaurant_features'] = data['restaurant_features'].apply(lambda x: bag_of_cuisines(x, restaurant_features))

cuisine_features.remove('set_init')
restaurant_features.remove('set_init')

In [94]:
cuisine_features = sorted((list(cuisine_features)))

In [115]:
restaurant_features = sorted((list(restaurant_features)))

Getting the dummies of the features to insert them into the machine learning model

In [97]:
cuisine_dummies = []
restaurant_dummies = []

def manual_dummies(bag, dummies, features):
  dummies.append([1 if i in bag else 0 for i in features])

data['cuisine'].apply(lambda x: manual_dummies(x, cuisine_dummies, cuisine_features))
data[cuisine_features] = cuisine_dummies
data.rename(columns={'Grill': 'Cuisine Grill'}, inplace=True)

data['restaurant_features'].apply(lambda x: manual_dummies(x, restaurant_dummies, restaurant_features))
data[restaurant_features] = restaurant_dummies
data.rename(columns={'Grill': 'Restaurant Feature Grill'}, inplace=True)

Now we get the dummies of the other categorical data we have

In [98]:
data.drop(columns=['cuisine', 'restaurant_features'], inplace=True)

y = data['Ratings']
data.drop(columns=['Ratings'], inplace=True)

In [99]:
area = data['area']
area_dummies = pd.get_dummies(area)

data.drop(columns=['area'], inplace=True)
data[area_dummies.columns] = area_dummies

Changing discount values from 0 and 1 to strings in order to create columns from

In [100]:
data['discounted'] = data['discounted'].apply(lambda x: 'Got Discount' if x else "Didn't Get Discount")
data.head(15)

,user_uuid,restaurant_uuid,discounted,nOrders,American,Argentinian,Asian,Bakeries and Pastries,Burgers,Chicken,Chinese,Coffee & Drinks,Desserts,Egyptian,European,Fast Food,Foul we Ta3meya,French,German,Cuisine Grill,Indian,International,Italian,Japanese,Koshary,Lebanese,Mexican,Oriental,Pizza,Sandwiches,Seafood,Spanish,Steaks,Sushi,Syrian,Tarts and chocolates,Tex-Mex,Thai,Turkish,Vegetarian,...,Ain Shams,Al Katameya,Al Mokatam,Al Raml Station,Al Rehab City,Al Shatby,Alexandria Desert Road,Camp Cesar,Cleopatra,Dokki,Downtown,El Sayeda Zeinab,Faisal,Garden City,Giza,Hadayek El Ahram,Hadayek El Kobba,Haram,Heliopolis,Imbaba,Kafr Abdo,Louran,Maadi,Madinaty,Manial,Masaken Sheraton,Misr El Kadima,Mohandessin,Moharam Bek,Nasr City,New Cairo,Ras Sudr,Semouha,Sheikh Zayed,Shoubra,Sidi Beshr Mohamed Naguib,Smart Village,Sporting,Stanley,Zamalek
0,4f0181917069f1bf5259c04f14a944c49d461bab,bf5ee7c8457db4d32da6633baf7a047115d76af4,Didn't Get Discount,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,06c7a3629566f04e19611981d0fb30dabe8d8daa,a8f1c42a5b27f032bbd69821b3d9941939db1ecf,Didn't Get Discount,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,966acbc0eaf5c2cda02a242ec9caf75f98119ef9,ef29eff86405a92b1b547b32e845497694f450c6,Didn't Get Discount,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,a910926f38c18c2e00a338c7d2f6e03ee78ea7a4,66ae2fbb6d1dfc1d3b07a68839db5da163eabe68,Didn't Get Discount,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,aa25b862b3f7fbd71425170343c53dcd93ec10c9,ef29eff86405a92b1b547b32e845497694f450c6,Didn't Get Discount,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,ffc49ace45603ad21123b88c06fc54cd00a48129,5f93fc73d2acda78d60df7926a929057315b8789,Didn't Get Discount,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,87ea8919aa62ba9c7357c6c0730d81a19c2ca98a,c3729414106b329a210c79d8b2752b83138be51d,Didn't Get Discount,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,127e3d20c2fdafb4759e99a7f4a0262fd0006848,94be8adc484693603f243907d964c8ad322a7788,Didn't Get Discount,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,c180723ead4054910ac50871b3cf5229e0531673,a8f1c42a5b27f032bbd69821b3d9941939db1ecf,Got Discount,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,30340a73025ddf1add5f3c24a0fbba1de32a3fd0,8d57fac7436e3448adf11840fb293567a75095ad,Didn't Get Discount,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [101]:
discount = data['discounted']
discount_dummies = pd.get_dummies(discount)

data.drop(columns=['discounted'], inplace=True)
data[discount_dummies.columns] = discount_dummies

In [102]:
data.head()

,user_uuid,restaurant_uuid,nOrders,American,Argentinian,Asian,Bakeries and Pastries,Burgers,Chicken,Chinese,Coffee & Drinks,Desserts,Egyptian,European,Fast Food,Foul we Ta3meya,French,German,Cuisine Grill,Indian,International,Italian,Japanese,Koshary,Lebanese,Mexican,Oriental,Pizza,Sandwiches,Seafood,Spanish,Steaks,Sushi,Syrian,Tarts and chocolates,Tex-Mex,Thai,Turkish,Vegetarian,24 hrs,...,Al Mokatam,Al Raml Station,Al Rehab City,Al Shatby,Alexandria Desert Road,Camp Cesar,Cleopatra,Dokki,Downtown,El Sayeda Zeinab,Faisal,Garden City,Giza,Hadayek El Ahram,Hadayek El Kobba,Haram,Heliopolis,Imbaba,Kafr Abdo,Louran,Maadi,Madinaty,Manial,Masaken Sheraton,Misr El Kadima,Mohandessin,Moharam Bek,Nasr City,New Cairo,Ras Sudr,Semouha,Sheikh Zayed,Shoubra,Sidi Beshr Mohamed Naguib,Smart Village,Sporting,Stanley,Zamalek,Didn't Get Discount,Got Discount
0,4f0181917069f1bf5259c04f14a944c49d461bab,bf5ee7c8457db4d32da6633baf7a047115d76af4,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,06c7a3629566f04e19611981d0fb30dabe8d8daa,a8f1c42a5b27f032bbd69821b3d9941939db1ecf,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,966acbc0eaf5c2cda02a242ec9caf75f98119ef9,ef29eff86405a92b1b547b32e845497694f450c6,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,a910926f38c18c2e00a338c7d2f6e03ee78ea7a4,66ae2fbb6d1dfc1d3b07a68839db5da163eabe68,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,aa25b862b3f7fbd71425170343c53dcd93ec10c9,ef29eff86405a92b1b547b32e845497694f450c6,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


Getting our data ready to insert it into our model

In [103]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [104]:
data['Ratings'] = y
data.head()

,user_uuid,restaurant_uuid,nOrders,American,Argentinian,Asian,Bakeries and Pastries,Burgers,Chicken,Chinese,Coffee & Drinks,Desserts,Egyptian,European,Fast Food,Foul we Ta3meya,French,German,Cuisine Grill,Indian,International,Italian,Japanese,Koshary,Lebanese,Mexican,Oriental,Pizza,Sandwiches,Seafood,Spanish,Steaks,Sushi,Syrian,Tarts and chocolates,Tex-Mex,Thai,Turkish,Vegetarian,24 hrs,...,Al Raml Station,Al Rehab City,Al Shatby,Alexandria Desert Road,Camp Cesar,Cleopatra,Dokki,Downtown,El Sayeda Zeinab,Faisal,Garden City,Giza,Hadayek El Ahram,Hadayek El Kobba,Haram,Heliopolis,Imbaba,Kafr Abdo,Louran,Maadi,Madinaty,Manial,Masaken Sheraton,Misr El Kadima,Mohandessin,Moharam Bek,Nasr City,New Cairo,Ras Sudr,Semouha,Sheikh Zayed,Shoubra,Sidi Beshr Mohamed Naguib,Smart Village,Sporting,Stanley,Zamalek,Didn't Get Discount,Got Discount,Ratings
0,4f0181917069f1bf5259c04f14a944c49d461bab,bf5ee7c8457db4d32da6633baf7a047115d76af4,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,5
1,06c7a3629566f04e19611981d0fb30dabe8d8daa,a8f1c42a5b27f032bbd69821b3d9941939db1ecf,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4
2,966acbc0eaf5c2cda02a242ec9caf75f98119ef9,ef29eff86405a92b1b547b32e845497694f450c6,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4
3,a910926f38c18c2e00a338c7d2f6e03ee78ea7a4,66ae2fbb6d1dfc1d3b07a68839db5da163eabe68,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,aa25b862b3f7fbd71425170343c53dcd93ec10c9,ef29eff86405a92b1b547b32e845497694f450c6,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4


Normalizing the data

In [105]:
scaler = StandardScaler()
data.iloc[:, 2:-1] = scaler.fit_transform(data.iloc[:, 2:-1])

In [106]:
data.head()

,user_uuid,restaurant_uuid,nOrders,American,Argentinian,Asian,Bakeries and Pastries,Burgers,Chicken,Chinese,Coffee & Drinks,Desserts,Egyptian,European,Fast Food,Foul we Ta3meya,French,German,Cuisine Grill,Indian,International,Italian,Japanese,Koshary,Lebanese,Mexican,Oriental,Pizza,Sandwiches,Seafood,Spanish,Steaks,Sushi,Syrian,Tarts and chocolates,Tex-Mex,Thai,Turkish,Vegetarian,24 hrs,...,Al Raml Station,Al Rehab City,Al Shatby,Alexandria Desert Road,Camp Cesar,Cleopatra,Dokki,Downtown,El Sayeda Zeinab,Faisal,Garden City,Giza,Hadayek El Ahram,Hadayek El Kobba,Haram,Heliopolis,Imbaba,Kafr Abdo,Louran,Maadi,Madinaty,Manial,Masaken Sheraton,Misr El Kadima,Mohandessin,Moharam Bek,Nasr City,New Cairo,Ras Sudr,Semouha,Sheikh Zayed,Shoubra,Sidi Beshr Mohamed Naguib,Smart Village,Sporting,Stanley,Zamalek,Didn't Get Discount,Got Discount,Ratings
0,4f0181917069f1bf5259c04f14a944c49d461bab,bf5ee7c8457db4d32da6633baf7a047115d76af4,-0.406276,-0.152025,-0.007937,-0.186139,-0.193653,-0.514340,-0.261767,-0.13622,-0.102548,-0.181154,1.609220,-0.047275,-0.858785,-0.197919,-0.041429,-0.072936,1.975575,-0.070631,-0.183478,-0.150062,-0.121089,-0.216474,-0.112672,-0.04578,1.814211,-0.244521,-0.791894,-0.077838,-0.016651,-0.14029,-0.147451,-0.370996,-0.027733,-0.081599,-0.048334,-0.047007,-0.202143,-0.507517,...,-0.028628,-0.238974,-0.031363,-0.008695,-0.007099,-0.021596,-0.158869,-0.115112,-0.018447,-0.060661,-0.024081,-0.024599,-0.031962,-0.065493,-0.081442,2.376011,-0.017025,-0.011773,-0.029282,-0.453571,-0.059924,-0.071881,-0.132478,-0.008695,-0.318951,-0.006148,-0.40163,-0.434088,-0.011773,-0.048985,-0.165336,-0.024081,-0.013748,-0.054956,-0.013748,-0.028407,-0.225875,0.55771,-0.55771,5
1,06c7a3629566f04e19611981d0fb30dabe8d8daa,a8f1c42a5b27f032bbd69821b3d9941939db1ecf,-0.406276,-0.152025,-0.007937,-0.186139,-0.193653,1.944239,-0.261767,-0.13622,-0.102548,-0.181154,-0.621419,-0.047275,1.164436,-0.197919,-0.041429,-0.072936,-0.506182,-0.070631,-0.183478,-0.150062,-0.121089,-0.216474,-0.112672,-0.04578,-0.551204,-0.244521,1.262795,-0.077838,-0.016651,-0.14029,-0.147451,-0.370996,-0.027733,-0.081599,-0.048334,-0.047007,-0.202143,1.970378,...,-0.028628,-0.238974,-0.031363,-0.008695,-0.007099,-0.021596,-0.158869,-0.115112,-0.018447,-0.060661,-0.024081,-0.024599,-0.031962,-0.065493,12.278633,-0.420873,-0.017025,-0.011773,-0.029282,-0.453571,-0.059924,-0.071881,-0.132478,-0.008695,-0.318951,-0.006148,-0.40163,-0.434088,-0.011773,-0.048985,-0.165336,-0.024081,-0.013748,-0.054956,-0.013748,-0.028407,-0.225875,0.55771,-0.55771,4
2,966acbc0eaf5c2cda02a242ec9caf75f98119ef9,ef29eff86405a92b1b547b32e845497694f450c6,1.834134,-0.152025,-0.007937,-0.186139,-0.193653,-0.514340,-0.261767,-0.13622,-0.102548,-0.181154,1.609220,-0.047275,-0.858785,-0.197919,-0.041429,-0.072936,1.975575,-0.070631,-0.183478,-0.150062,-0.121089,-0.216474,-0.112672,-0.04578,1.814211,-0.244521,-0.791894,-0.077838,-0.016651,-0.14029,-0.147451,-0.370996,-0.027733,-0.081599,-0.048334,-0.047007,-0.202143,-0.507517,...,-0.028628,-0.238974,-0.031363,-0.008695,-0.007099,-0.021596,-0.158869,-0.115112,-0.018447,-0.060661,-0.024081,-0.024599,-0.031962,-0.065493,-0.081442,-0.420873,-0.017025,-0.011773,-0.029282,2.204725,-0.059924,-0.071881,-0.132478,-0.008695,-0.318951,-0.006148,-0.40163,-0.434088,-0.011773,-0.048985,-0.165336,-0.024081,-0.013748,-0.054956,-0.013748,-0.028407,-0.225875,0.55771,-0.55771,4
3,a910926f38c18c2e00a338c7d2f6e03ee78ea7a4,66ae2fbb6d1dfc1d3b07a68839db5da163eabe68,-0.406276,-0.152025,-0.007937,-0.186139,-0.193653,-0.514340,-0.261767,-0.13622,-0.102548,-0.181154,-0.621419,-0.047275,1.164436,-0.197919,-0.041429,-0.072936,-0.506182,-0.070631,-0.183478,-0.150062,-0.121089,-0.216474,-0.112672,-0.04578,-0.551204,4.089633,-0.791894,-0.077838,-0.016651,-0.14029,-0.147451,-0.370996,-0.027733,-0.081599,-0.048334,-0.047007,-0.202143,1.970378,...,-0.028628,-0.238974,-0.031363,-0.008695,-0.007099,-0.021596,-0.158869,-0.115112,-0.018447,16.485128,-0.024081,-0.024599,-0.031962,-0.065493,-

In [107]:
X_train, X_test, y_train, y_test = train_test_split(
    data.iloc[:, 2:-1].values, data.iloc[:, -1].values, test_size = 0.2, random_state = 0)

We will use decision tree classification since the ratings are from 1-5

In [108]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

calculating F1 score for accuracy

In [114]:
from sklearn.metrics import f1_score
f1_score(y_test, model.predict(X_test), average='micro')

0.9988031496062992